In [ ]:
import pandas as pd
import numpy
from tqdm import tqdm
tqdm.pandas()

# GETTING MEAS

In [ ]:
meas=pd.read_csv('meas2.csv',header=0)
meas['val'] = pd.to_numeric(meas['val'], errors='coerce')
pt=pd.read_csv('patientVisitAge.csv',header=0)
meas=pd.merge(meas,pt[['person_id','visit_occurrence_id','Age']],on=['person_id','visit_occurrence_id'],how='left')
meas=meas.dropna()

In [ ]:
meas.head()

In [ ]:
meas['Age']=meas['Age']*12
meas['Age']=meas['Age'].astype(int)
meas=meas[meas['Age']<=120]

In [ ]:
bmi=pd.read_csv('bmip_signal.csv',header=0)
meas=meas[meas['person_id'].isin(bmi['person_id'])]

In [ ]:
meas['person_id'].nunique()

In [ ]:
meas.fact_id.nunique()

In [ ]:
meas_ids=[3013762,
 3038553,
 3023540,
 40762638,
 2000000043,
 3020891,
 3001537,
 3000963,
 3028406,
 3027018,
 3024171,
 3004249,
 3012888,
 3009542,
 3024731,
 3012030,
 3010813,
 3009744,
 3010910,
 3022174,
 3007461,
 3049383,
 3045424,
 3002030,
 40762499,
 3020951,
 3017354,
 3019069,
 3017732,
 3020416,
 3037577,
 3019198,
 3001604,
 3006504,
 3022640,
 3019550,
 3014576,
 3006906,
 3013682,
 3004501,
 3016723,
 3043111,
 3015632,
 3013115,
 3023103,

 3024128,
 3022621,
 3000330,
 3006315,
 3024629,
 3007950,

 
 3031147,
 
 3024561,
 3035995,
 3020630,
 3006923]

In [ ]:
meas_ids=[
 2000000043,
 3020891,
 3000963,
 3028406,
 3027018,
 3024171,
 3004249,
 3012888,
 3009542,
 3024731,
 3012030,
 3010813,
 3009744,
 3010910,
 3022174,
 3007461,
 3049383,
 3045424,
 3002030,
 40762499,
 3020951,
 3017354,
 3019069,
 3017732,
 3020416,
 3037577,
 3019198,
 3001604,
 3006504,
 3022640,
 3019550,
 3014576,
 3006906,
 3013682,
 3004501,
 3016723,
 3043111,
 3015632,
 3013115,
 3023103,

 3024128,
 3022621,
 3000330,
 3006315,
 3024629,
 3007950,

 
 3031147,
 
 3024561,
 3035995,
 3020630,
 3006923]

In [ ]:
meas=meas[meas['fact_id'].isin(meas_ids)]

In [ ]:
meas['fact_id'].nunique()

In [ ]:
meas_dict=dict(zip(meas['fact_id'],meas['fact']))
import pickle
with open("data/5/measVocab", 'wb') as fp:
        pickle.dump(meas_dict, fp)

In [ ]:
meas_dict

In [ ]:
meas['fact'].unique()

In [ ]:
meas.head()

In [ ]:
meas['fact_id']=meas['fact_id'].astype(int)

In [ ]:
from tqdm import tqdm
import numpy as np
t=3
feat=meas['fact_id'].unique()

pids=meas['person_id'].unique()
final_labs=pd.DataFrame()
for i in tqdm(range(0,12,3)): 
    sub_labs=meas[(meas['Age']>=i) & (meas['Age']<i+3)].groupby(['person_id','fact_id']).agg({'val':'median'})
    sub_labs=sub_labs.reset_index()
    sub_labs=sub_labs.pivot_table(index='person_id',columns='fact_id',values='val')
    
    sub_labs=sub_labs.reset_index()
    sub_labs.columns.name=None
    sub_labs=sub_labs.reset_index(drop=True)
    feat_df=pd.DataFrame(columns=list(set(feat)-set(list(sub_labs.columns))))
    if not feat_df.empty:
        sub_labs=pd.concat([sub_labs,feat_df],axis=1)
    
    pid_col=sub_labs['person_id']
    sub_labs=sub_labs[feat]
    #print(sub_labs.head())
    #print(pid_col.head())
    sub_labs=pd.concat([sub_labs,pid_col],axis=1)
    #print(sub_labs.head())
    pid_df=pd.DataFrame(columns=feat)
    #print(list(set(pids)-set(sub_labs['person_id'].unique())))
    temp=pd.Series(list(set(pids)-set(sub_labs['person_id'].unique())),name='person_id')
    #print(temp)
    for _ in range(temp.shape[0]):
        pid_df = pid_df.append(pd.Series(), ignore_index=True)
    pid_df=pd.concat([pid_df,temp],axis=1)
    #print(pid_df.head())
    sub_labs=pd.concat([sub_labs,pid_df],axis=0)
    
    sub_labs['Age']=t
    print(sub_labs.shape)
    #print(sub_labs.head())
    if final_labs.empty:
        final_labs=sub_labs
    else:    
        final_labs=final_labs.append(sub_labs)
    
    t=t+3

In [ ]:
from tqdm import tqdm
import numpy as np
t=18
feat=meas['fact_id'].unique()

pids=meas['person_id'].unique()
final_labs2=pd.DataFrame()
for i in tqdm(range(12,24,6)): 
    sub_labs=meas[(meas['Age']>=i) & (meas['Age']<i+3)].groupby(['person_id','fact_id']).agg({'val':'median'})
    
    sub_labs=sub_labs.reset_index()
    sub_labs=sub_labs.pivot_table(index='person_id',columns='fact_id',values='val')
    
    sub_labs=sub_labs.reset_index()
    sub_labs.columns.name=None
    sub_labs=sub_labs.reset_index(drop=True)
    feat_df=pd.DataFrame(columns=list(set(feat)-set(list(sub_labs.columns))))
    if not feat_df.empty:
        sub_labs=pd.concat([sub_labs,feat_df],axis=1)
    
    pid_col=sub_labs['person_id']
    sub_labs=sub_labs[feat]
    #print(sub_labs.head())
    #print(pid_col.head())
    sub_labs=pd.concat([sub_labs,pid_col],axis=1)
    #print(sub_labs.head())
    pid_df=pd.DataFrame(columns=feat)
    #print(list(set(pids)-set(sub_labs['person_id'].unique())))
    temp=pd.Series(list(set(pids)-set(sub_labs['person_id'].unique())),name='person_id')
    #print(temp)
    for _ in range(temp.shape[0]):
        pid_df = pid_df.append(pd.Series(), ignore_index=True)
    pid_df=pd.concat([pid_df,temp],axis=1)
    #print(pid_df.head())
    sub_labs=pd.concat([sub_labs,pid_df],axis=0)
    
    sub_labs['Age']=t
    print(sub_labs.shape)
    #print(sub_labs.head())
    if final_labs.empty:
        final_labs2=sub_labs
    else:    
        final_labs2=final_labs2.append(sub_labs)
    
    t=t+6

In [ ]:
final_input=pd.concat([final_labs,final_labs2],axis=0)

In [ ]:
final_input.head()

In [ ]:
bmi=final_input[['person_id','Age',3001537,3013762,3023540]]
del final_input[3001537]
del final_input[3013762]
del final_input[3023540]

In [ ]:
final_input=final_input.sort_values(by=['person_id','Age'])
bmi=bmi.sort_values(by=['person_id','Age'])

In [ ]:
bmi.head()

In [ ]:
bmi=bmi.pivot_table(index='person_id',columns='Age',values='label')
bmi=bmi.reset_index()
bmi.head()

In [ ]:
bmi.replace(0, np.nan, inplace=True)
bmi.head()

In [ ]:
bmi=bmi.bfill(axis=1).ffill(axis=1)
bmi.head()

In [ ]:
bmi.to_csv('bmi_02.csv',index=False)

In [ ]:
bmi_pct=bmi.iloc[:,1:].pct_change(axis='columns')

In [ ]:
bmi_pct=pd.concat([bmi['person_id'],bmi_pct],axis=1)

In [ ]:
bmi_pct.to_csv('bmi_pct_02.csv',index=False)

In [ ]:
bmi_pct=pd.read_csv('data/4/bmi_pct_02.csv',header=0)

In [ ]:
bmi_pct.head()

In [ ]:
del bmi_pct['3']
cols_to_std=list(bmi_pct.columns)
cols_to_std=cols_to_std[1:]
cols_to_std

In [ ]:
for index in range(len(cols_to_std)):
    bmi_pct[cols_to_std[index]], qcut_intervals = pd.qcut(bmi_pct[cols_to_std[index]], 5, labels=False, duplicates='drop',retbins=True)
    print(qcut_intervals)

In [ ]:
qcut_intervals

In [ ]:
del bmi_pct[24]

In [ ]:
bmi_pct

In [ ]:
bmi_pct

In [ ]:
bmi_pct1=pd.melt(bmi_pct, id_vars=['person_id'], value_vars=[6, 9, 12, 18])

In [ ]:
bmi_pct1=bmi_pct1.sort_values(by=['person_id','variable'])

In [ ]:
bmi_pct1.to_csv('bmi_pct_enc_02.csv',index=False)

In [ ]:
cols_to_std=list(final_input.columns)
cols_to_std=cols_to_std[:len(cols_to_std)-2]

In [ ]:
for index in range(len(cols_to_std)):
    final_input[cols_to_std[index]], qcut_intervals = pd.qcut(final_input[cols_to_std[index]], 5, labels=False, duplicates='drop',retbins=True)

In [ ]:
final_input

In [ ]:
final_input['person_id'].nunique()

In [ ]:
final_input1=pd.melt(final_input, id_vars=['person_id','Age'], value_vars=cols_to_std)

In [ ]:
final_input1['person_id'].nunique()

In [ ]:
final_input1.head()

In [ ]:
final_input1=final_input1.dropna()

In [ ]:
final_input1=final_input1.sort_values(by=['person_id','Age'])


In [ ]:
final_input1['value']=final_input1['variable'].astype(str)+str('_')+final_input1['value'].astype(int).astype(str)
final_input1.head()

In [ ]:
final_input1.groupby('person_id').count()['Age'].max()

In [ ]:
final_input1.groupby('person_id').count()['Age'].mean()

In [ ]:
final_input1.groupby('person_id').count()['Age'].std()

In [ ]:
final_input1=final_input1.reset_index(drop=True)

In [ ]:
final_input1.person_id.nunique()

In [ ]:
final_input1.to_csv('data/4/meas_02_5.csv',index=False)

### BMI SIGNAL AFTER 2

In [ ]:
meas=pd.read_csv('meas2.csv',header=0)
meas['val'] = pd.to_numeric(meas['val'], errors='coerce')
pt=pd.read_csv('patientVisitAge.csv',header=0)
meas=pd.merge(meas,pt[['person_id','visit_occurrence_id','Age']],on=['person_id','visit_occurrence_id'],how='left')
meas=meas.dropna()
meas['Age']=meas['Age']*12
meas['Age']=meas['Age'].astype(int)
meas=meas[meas['Age']>24]
meas=meas[meas['Age']<=120]

meas=meas[meas['fact_id'].isin(['person_id','Age',3013762,3023540])]

In [ ]:
meas.head()

In [ ]:
from tqdm import tqdm
import numpy as np
t=36
feat=meas['fact_id'].unique()

pids=meas['person_id'].unique()
final_labs=pd.DataFrame()
for i in tqdm(range(24,120,12)): 
    sub_labs=meas[(meas['Age']>=i) & (meas['Age']<i+12)].groupby(['person_id','fact_id']).agg({'val':'median'})
    sub_labs=sub_labs.reset_index()
    sub_labs=sub_labs.pivot_table(index='person_id',columns='fact_id',values='val')
    
    sub_labs=sub_labs.reset_index()
    sub_labs.columns.name=None
    sub_labs=sub_labs.reset_index(drop=True)
    feat_df=pd.DataFrame(columns=list(set(feat)-set(list(sub_labs.columns))))
    if not feat_df.empty:
        sub_labs=pd.concat([sub_labs,feat_df],axis=1)
    
    pid_col=sub_labs['person_id']
    sub_labs=sub_labs[feat]
    #print(sub_labs.head())
    #print(pid_col.head())
    sub_labs=pd.concat([sub_labs,pid_col],axis=1)
    #print(sub_labs.head())
    pid_df=pd.DataFrame(columns=feat)
    #print(list(set(pids)-set(sub_labs['person_id'].unique())))
    temp=pd.Series(list(set(pids)-set(sub_labs['person_id'].unique())),name='person_id')
    #print(temp)
    for _ in range(temp.shape[0]):
        pid_df = pid_df.append(pd.Series(), ignore_index=True)
    pid_df=pd.concat([pid_df,temp],axis=1)
    #print(pid_df.head())
    sub_labs=pd.concat([sub_labs,pid_df],axis=0)
    
    sub_labs['Age']=t
    print(sub_labs.shape)
    #print(sub_labs.head())
    if final_labs.empty:
        final_labs=sub_labs
    else:    
        final_labs=final_labs.append(sub_labs)
    
    t=t+12

In [ ]:
final_labs.head()

In [ ]:
final_labs=final_labs[['person_id','Age','BMI','label']]

In [ ]:
final_labs.to_csv('data/4/rawbmi_after2.csv',index=False)

In [ ]:
ob=final_labs[['person_id','Age','label']]
ob['ob_label']=0
ob['over_label']=0
ob.loc[ob['label']>=95,'ob_label']=1
ob.loc[(ob['label']>=85) & (ob['label']<95),'over_label']=1
ob.head()

In [ ]:
bmi=final_labs.pivot_table(index='person_id',columns='Age',values='label')
bmi=bmi.reset_index()
bmi.head()

In [ ]:
bmi_pad=bmi.iloc[:,1:].interpolate(method='linear', limit_direction='both', limit_area='inside',axis=1)

In [ ]:
bmi_mask=bmi_pad.fillna(0)
bmi_mask[bmi_mask>0]=1
bmi_mask

In [ ]:
bmi_ob=bmi_pad.fillna(0)
bmi_ob[bmi_ob<95]=0
bmi_ob[bmi_ob>=95]=1
bmi_ob

In [ ]:
bmi_pad=pd.concat([bmi[['person_id']],bmi_pad],axis=1)
bmi_pad

In [ ]:
bmi_pad.to_csv('data/4/bmiPad_after2.csv',index=False)

In [ ]:
bmi_mask=pd.concat([bmi[['person_id']],bmi_mask],axis=1)
bmi_mask.to_csv('data/4/bmiMask_after2.csv',index=False)

In [ ]:
bmi_ob=pd.concat([bmi[['person_id']],bmi_ob],axis=1)
bmi_ob.to_csv('data/4/bmiOb_after2.csv',index=False)

### MEAS AFTER 2

In [ ]:
meas=pd.read_csv('meas2.csv',header=0)
meas['val'] = pd.to_numeric(meas['val'], errors='coerce')
pt=pd.read_csv('patientVisitAge.csv',header=0)
meas=pd.merge(meas,pt[['person_id','visit_occurrence_id','Age']],on=['person_id','visit_occurrence_id'],how='left')
meas=meas.dropna()
meas['Age']=meas['Age']*12
meas['Age']=meas['Age'].astype(int)
meas=meas[meas['Age']>24]
meas=meas[meas['Age']<=120]

meas=meas[meas['fact_id'].isin(meas_ids)]

In [ ]:
from tqdm import tqdm
import numpy as np
t=36
feat=meas['fact_id'].unique()

pids=meas['person_id'].unique()
final_labs=pd.DataFrame()
for i in tqdm(range(24,120,12)): 
    sub_labs=meas[(meas['Age']>=i) & (meas['Age']<i+12)].groupby(['person_id','fact_id']).agg({'val':'median'})
    sub_labs=sub_labs.reset_index()
    sub_labs=sub_labs.pivot_table(index='person_id',columns='fact_id',values='val')
    
    sub_labs=sub_labs.reset_index()
    sub_labs.columns.name=None
    sub_labs=sub_labs.reset_index(drop=True)
    feat_df=pd.DataFrame(columns=list(set(feat)-set(list(sub_labs.columns))))
    if not feat_df.empty:
        sub_labs=pd.concat([sub_labs,feat_df],axis=1)
    
    pid_col=sub_labs['person_id']
    sub_labs=sub_labs[feat]
    #print(sub_labs.head())
    #print(pid_col.head())
    sub_labs=pd.concat([sub_labs,pid_col],axis=1)
    #print(sub_labs.head())
    pid_df=pd.DataFrame(columns=feat)
    #print(list(set(pids)-set(sub_labs['person_id'].unique())))
    temp=pd.Series(list(set(pids)-set(sub_labs['person_id'].unique())),name='person_id')
    #print(temp)
    for _ in range(temp.shape[0]):
        pid_df = pid_df.append(pd.Series(), ignore_index=True)
    pid_df=pd.concat([pid_df,temp],axis=1)
    #print(pid_df.head())
    sub_labs=pd.concat([sub_labs,pid_df],axis=0)
    
    sub_labs['Age']=t
    print(sub_labs.shape)
    #print(sub_labs.head())
    if final_labs.empty:
        final_labs=sub_labs
    else:    
        final_labs=final_labs.append(sub_labs)
    
    t=t+12

In [ ]:
final_labs=final_labs.sort_values(by=['person_id','Age'])
final_labs

In [ ]:
cols_to_std=list(final_labs.columns)
cols_to_std=cols_to_std[:len(cols_to_std)-2]

In [ ]:
for index in range(len(cols_to_std)):
    final_labs[cols_to_std[index]], qcut_intervals = pd.qcut(final_labs[cols_to_std[index]], 5, labels=False, duplicates='drop',retbins=True)

In [ ]:
del final_labs[3001537]
del final_labs[3013762]
del final_labs[3023540]

In [ ]:
cols_to_std=list(final_labs.columns)
cols_to_std=cols_to_std[:len(cols_to_std)-2]

In [ ]:
final_input1=pd.melt(final_labs, id_vars=['person_id','Age'], value_vars=cols_to_std)
final_input1

In [ ]:
final_input1=final_input1.fillna(-1)
extra=final_input1[final_input1['value']==-1]
final_input1=final_input1[final_input1['value']>-1]
extra.loc[extra['value']==-1,'value']=0

In [ ]:
final_input1['value']=final_input1['variable'].astype(str)+str('_')+final_input1['value'].astype(int).astype(str)
final_input1.head()

In [ ]:
final_input1=pd.concat([final_input1,extra],axis=0)
final_input1

In [ ]:
final_input1=final_input1.sort_values(by=['person_id','Age'])
final_input1.head()

In [ ]:
final_input1.value.nunique()

In [ ]:
final_input1=final_input1.drop_duplicates()

In [ ]:
final_input2=final_input1.pivot_table(index=['person_id','Age'],columns='variable',values='value', aggfunc='first')
final_input2=final_input2.reset_index()
final_input2.head()

In [ ]:
final_input2=final_input2.fillna(0)
final_input2=final_input2.sort_values(by=['person_id','Age'])
final_input2.head()

In [ ]:
final_input2.to_csv('data/4/meas_after2.csv',index=False)

In [ ]:
final_input2.shape

In [ ]:
final_input2.person_id.nunique()

In [ ]:
train_id=pd.read_csv('data/4/train_ids.csv',header=0)
test_id=pd.read_csv('data/4/test_ids.csv',header=0)

In [ ]:
train_id.shape

In [ ]:
45600+14157

In [ ]:
test_id.shape

### Cond for decoder

In [ ]:
cond=pd.read_csv('cond_cohort.csv',header=0)
cond=cond[cond['Age']<=120]
cond['val']=1
del cond['visit_occurrence_id']
del cond['fact_id']
del cond['Sex']
cond=cond.drop_duplicates()
cond.head()

In [ ]:
cond.fact.unique()

In [ ]:
print(cond.shape)
print(cond.person_id.nunique())
print(cond.fact.nunique())

In [ ]:
146-67

In [ ]:
fh=[  'FH: Diabetes mellitus',
	   'FH: Cardiovascular disease',
       'Family history of asthma',
       'Family history of genetic disorder carrier',
       'FH: Gastrointestinal disease', 
	   'FH: Thrombosis',
       'FH: Drug dependency',
       'Family history of cardiovascular disease in first degree male relative less than 55 years of age',
       'FH: Myocardial infarction',
       'Family history of sudden cardiac death',
       'Family history of kidney disease',
       'Family history of chronic renal impairment',
       'Family history of neurological disorder',
       'Family history of cancer', 
	   'FH: Hypercholesterolemia',
       'Family history of autism',
       'Family history of ischemic heart disease',
       'Family history of mental disorder',
       'Family history of learning disability', 
	   'FH: Depression',
       'Family history of victim of physical abuse',
       'Family history of problem behavior',
       'Family history of thromboembolic disorder',
       'Family history of development disorder', 
	   'FH: Thyroid disorder',
       'FH: Muscular dystrophy',
       'Family history of diabetes mellitus type 1',
       'Family history of hyperlipidemia', 
	   'Family history of alcoholism',
       'FH: Cardiac disorder',
       'Family history of intellectual disability',
       'Family history of disorder of pancreas',
       'Family history of blood coagulation disorder',
       'Family history of malignant neoplasm of thyroid',
       'Family history of attention deficit hyperactivity disorder',
       'Family history of substance abuse',
	   'Family history of stroke',
       'Family history of Von Willebrand disease',
       'Family history of heart failure',
       'Family history of autism in sibling', 
	   'FH: Obesity',
       'Family history of conduction disorder of the heart',
       'FH: Hypertension', 
	   'Family history of consanguinity',
       'Family history of aneurysm of thoracic aorta',
       'FH: Neoplasm of CNS',
	   'FH: Cardiomyopathy',
       'Family history of polycystic ovary syndrome',
       'Family history of coronary arteriosclerosis',
       'FH: Diabetes mellitus in first degree relative',
       'FH: premature coronary heart disease', 
	   'FH: Hypothyroidism',
       'Family history of neoplasm',
	   'FH: Obstetric problem',
       'Family history of Graves disease', 
	   'FH: Manic-depressive state',
       'Family history of Hashimoto thyroiditis',
       'FH: Raised blood lipids',
	   'Family history of Marfan syndrome',
       'Family history of cardiac arrhythmia',
       'Family history of stenosis of aortic valve',
       'Family history of fragile X syndrome',
       'Family history of diabetes mellitus type 2',
       'Family history of smoking', 
	   'FH: Chromosomal anomaly',
       'Family history of aneurysm of artery',
       'FH: Diabetes in pregnancy', ]

In [ ]:
len(fh)

In [ ]:
from tqdm import tqdm
import numpy as np
t=36
feat=cond['fact'].unique()

pids=cond['person_id'].unique()
final_conds2=pd.DataFrame()
for i in tqdm(range(24,120,12)): 
    sub_labs=cond[(cond['Age']>=i) & (cond['Age']<i+12)].groupby(['person_id','fact']).agg({'val':np.max})
    sub_labs=sub_labs.reset_index()
    sub_labs=sub_labs.pivot_table(index='person_id',columns='fact',values='val')
    
    sub_labs=sub_labs.reset_index()
    sub_labs.columns.name=None
    sub_labs=sub_labs.reset_index(drop=True)
    #print(sub_labs.shape)
    #print(list(set(feat)-set(list(sub_labs.columns))))
    feat_df=pd.DataFrame(columns=list(set(feat)-set(list(sub_labs.columns))))
    #print(feat_df.shape)
    if feat_df.shape[1]!=0:
        #print("hi")
        sub_labs=pd.concat([sub_labs,feat_df],axis=1)
    #print(sub_labs.shape)
    pid_col=sub_labs['person_id']
    #print(sub_labs)
    sub_labs=sub_labs[feat]
    #print(sub_labs.head())
    #print(pid_col.head())
    sub_labs=pd.concat([sub_labs,pid_col],axis=1)
    #print(sub_labs.head())
    pid_df=pd.DataFrame(columns=feat)
    #print(list(set(pids)-set(sub_labs['person_id'].unique())))
    temp=pd.Series(list(set(pids)-set(sub_labs['person_id'].unique())),name='person_id')
    #print(temp)
    for _ in range(temp.shape[0]):
        pid_df = pid_df.append(pd.Series(), ignore_index=True)
    pid_df=pd.concat([pid_df,temp],axis=1)
    #print(pid_df.head())
    sub_labs=pd.concat([sub_labs,pid_df],axis=0)
    
    sub_labs['Age']=t
    print(sub_labs.shape)
    #print(sub_labs.head())
    if final_conds2.empty:
        final_conds2=sub_labs
    else:    
        final_conds2=final_conds2.append(sub_labs)
    
    t=t+12

In [ ]:
final_conds2.to_csv('data/4/cond_after2.csv',index=False)

In [ ]:
final_conds2=pd.read_csv('data/4/final_cond.csv',header=0)

In [ ]:
final_conds2=final_conds2.fillna(0)
final_conds2=final_conds2.sort_values(by=['person_id','Age'])
final_conds2.head()

In [ ]:
print(final_conds2.shape)
print(final_conds2.person_id.nunique())

### MED for decoder

In [ ]:
med=pd.read_csv('ATC10.csv',header=0)
med=med[med['Age']<=120]
med['val']=1
med=med[['person_id','mapped_concept_name','Age','val']]
med=med.drop_duplicates()
med.head()

In [ ]:
med['mapped_concept_name'].nunique()

In [ ]:
med['Age'].max()

In [ ]:
temp=med[med['Age']<=24]
del temp['Age']
temp=temp.drop_duplicates()
temp=temp.groupby(['person_id','mapped_concept_name']).count()
temp=temp.reset_index()
temp.head()

In [ ]:
temp.shape

In [ ]:
temp=temp.sort_values(by='val',ascending=False)
#temp['val']=temp['val']/temp.shape[0]

In [ ]:
temp['val'].mean()

In [ ]:
temp['val'].max()

In [ ]:
temp[temp['val']>=1396].shape

In [ ]:
temp[temp['val']>=1396]['mapped_concept_name']

In [ ]:
print(med.shape)
print(med.person_id.nunique())
print(med.mapped_concept_name.nunique())

In [ ]:
from tqdm import tqdm
import numpy as np
t=36
feat=med['mapped_concept_name'].unique()

pids=med['person_id'].unique()
final_conds2=pd.DataFrame()
for i in tqdm(range(24,120,12)): 
    sub_labs=med[(med['Age']>=i) & (med['Age']<i+12)].groupby(['person_id','mapped_concept_name']).agg({'val':np.max})
    sub_labs=sub_labs.reset_index()
    sub_labs=sub_labs.pivot_table(index='person_id',columns='mapped_concept_name',values='val')
    print(sub_labs.shape)
    sub_labs=sub_labs.reset_index()
    sub_labs.columns.name=None
    sub_labs=sub_labs.reset_index(drop=True)
    #print(sub_labs.shape)
    #print(list(set(feat)-set(list(sub_labs.columns))))
    feat_df=pd.DataFrame(columns=list(set(feat)-set(list(sub_labs.columns))))
    #print(feat_df.shape)
    if feat_df.shape[1]!=0:
        #print("hi")
        sub_labs=pd.concat([sub_labs,feat_df],axis=1)
    print(sub_labs.shape)
    pid_col=sub_labs['person_id']
    #print(sub_labs)
    sub_labs=sub_labs[feat]
    #print(sub_labs.head())
    #print(pid_col.head())
    sub_labs=pd.concat([sub_labs,pid_col],axis=1)
    #print(sub_labs.head())
    pid_df=pd.DataFrame(columns=feat)
    #print(list(set(pids)-set(sub_labs['person_id'].unique())))
    temp=pd.Series(list(set(pids)-set(sub_labs['person_id'].unique())),name='person_id')
    #print(temp)
    for _ in range(temp.shape[0]):
        pid_df = pid_df.append(pd.Series(), ignore_index=True)
    pid_df=pd.concat([pid_df,temp],axis=1)
    #print(pid_df.head())
    sub_labs=pd.concat([sub_labs,pid_df],axis=0)
    
    sub_labs['Age']=t
    print(sub_labs.shape)
    #print(sub_labs.head())
    if final_conds2.empty:
        final_conds2=sub_labs
    else:    
        final_conds2=final_conds2.append(sub_labs)
    
    t=t+12

In [ ]:
final_conds2.to_csv('data/5/meas_after2.csv',index=False)

In [ ]:
final_conds2=final_conds2.fillna(0)
final_conds2=final_conds2.sort_values(by=['person_id','Age'])
final_conds2.head()

In [ ]:
print(final_conds2.shape)
print(final_conds2.person_id.nunique())

In [ ]:
final_conds2

# DEMO

In [ ]:
demo=pd.read_csv('data/4/demo3.csv',header=0)
demo=demo.groupby('person_id').first()
demo=demo.reset_index()

In [ ]:
demo.head()

In [ ]:
payer=pd.read_csv('data/4/payer.csv',header=0)
payer.loc[payer['payer']=='Other/Unknown','payer']='NI'
payer.loc[payer['payer']=="['Medicaid/sCHIP' 'Private/Commercial']",'payer']='NI'
payer.loc[payer['payer']=="Medicare",'payer']='NI'

In [ ]:
payer.payer.value_counts()

In [ ]:
demo=pd.merge(demo,payer,on='person_id',how='left')
demo.to_csv('data/4/demo.csv',index=False)

### COI

In [ ]:
coi=pd.read_csv('coi.csv',header=0)
final_ids=pd.read_csv('data/4/tt/all_id.csv',header=0)
demo=pd.read_csv('data/4/tt/demo_all.csv',header=0)
coi=coi[coi['person_id'].isin(final_ids['person_id'].unique())]
coi['coi'], qcut_intervals = pd.qcut(coi['coi'], 10, labels=False, duplicates='drop',retbins=True)
coi['coi']='COI_'+coi['coi'].astype(str)
coi.head()

In [ ]:
coi=pd.read_csv('coi.csv',header=0)
final_ids=pd.read_csv('data/4/tt/all_id.csv',header=0)
demo=pd.read_csv('data/4/tt/demo_all.csv',header=0)
coi=coi[coi['person_id'].isin(final_ids['person_id'].unique())]
coi.head()

In [ ]:
coi['coi'].max()

In [ ]:
coi['coi'].mean()

In [ ]:
coi[(coi['coi']>=1) & (coi['coi']<=4)].shape

In [ ]:
coi[(coi['coi']>=4) & (coi['coi']<=13)].shape

In [ ]:
coi[(coi['coi']>=13) & (coi['coi']<=23)].shape

In [ ]:
coi[(coi['coi']>=89)].shape

In [ ]:
coi.shape

In [ ]:
qcut_intervals

In [ ]:
demo=pd.merge(demo,coi,how='inner',on='person_id')
demo=demo.sort_values(by='person_id')
demo.to_csv('data/5/tt/demo_all.csv',index=False)

### Statistics

In [ ]:
train_id=pd.read_csv('data/4/tt/train_id.csv',header=0)
test_id=pd.read_csv('data/4/tt/test_id.csv',header=0)
final_ids=pd.read_csv('data/4/tt/all_id.csv',header=0)
train_ids=train_id['person_id'].unique()
test_ids=test_id['person_id'].unique()
all_ids=final_ids['person_id'].unique()

In [ ]:
cond=pd.read_csv('cond_cohort.csv',header=0)
cond=cond[cond['Age']<=24]
del cond['visit_occurrence_id']
del cond['fact_id']
del cond['Sex']
cond=cond.drop_duplicates()
cond=cond[cond['person_id'].isin(all_ids)]
print("All Before 2 max", cond.groupby('person_id').count()['Age'].max())
print("All Before 2 mean", cond.groupby('person_id').count()['Age'].mean())
print("All Before 2 min", cond.groupby('person_id').count()['Age'].min())
cond1=cond[cond['person_id'].isin(train_ids)]
print("Train Before 2 max", cond1.groupby('person_id').count()['Age'].max())
print("Train Before 2 mean", cond1.groupby('person_id').count()['Age'].mean())
print("Train Before 2 std", cond1.groupby('person_id').count()['Age'].std())
print("Train Before 2 min", cond1.groupby('person_id').count()['Age'].min())
cond1=cond[cond['person_id'].isin(test_ids)]
print("Test Before 2 max", cond1.groupby('person_id').count()['Age'].max())
print("Test Before 2 mean", cond1.groupby('person_id').count()['Age'].mean())
print("Test Before 2 std", cond1.groupby('person_id').count()['Age'].std())
print("Test Before 2 min", cond1.groupby('person_id').count()['Age'].min())


cond=pd.read_csv('cond_cohort.csv',header=0)
cond=cond[cond['Age']>24]
cond=cond[cond['Age']<=120]
del cond['visit_occurrence_id']
del cond['fact_id']
del cond['Sex']
cond=cond.drop_duplicates()
cond=cond[cond['person_id'].isin(all_ids)]
print("All After 2 max", cond.groupby('person_id').count()['Age'].max())
print("All After 2 mean", cond.groupby('person_id').count()['Age'].mean())
print("All After 2 min", cond.groupby('person_id').count()['Age'].min())
cond1=cond[cond['person_id'].isin(train_ids)]
print("Train After 2 max", cond1.groupby('person_id').count()['Age'].max())
print("Train After 2 mean", cond1.groupby('person_id').count()['Age'].mean())
print("Train After 2 STd", cond1.groupby('person_id').count()['Age'].std())
print("Train After 2 min", cond1.groupby('person_id').count()['Age'].min())
cond1=cond[cond['person_id'].isin(test_ids)]
print("Test After 2 max", cond1.groupby('person_id').count()['Age'].max())
print("Test After 2 mean", cond1.groupby('person_id').count()['Age'].mean())
print("Test After 2 std", cond1.groupby('person_id').count()['Age'].std())
print("Test After 2 min", cond1.groupby('person_id').count()['Age'].min())

In [ ]:
cond=pd.read_csv('data/4/meas_02.csv',header=0)
print(cond.head())
del cond['value']
meas_ids=['3013762','3023540','40762638','3001537','3038553']
cond=cond[cond['variable'].isin(meas_ids)]
cond=cond[cond['Age']<=24]
#del cond['Age']
cond=cond.drop_duplicates()
cond=cond[cond['person_id'].isin(all_ids)]
print("All Before 2 max", cond.groupby('person_id').count()['Age'].max())
print("All Before 2 mean", cond.groupby('person_id').count()['Age'].mean())
print("All Before 2 std", cond.groupby('person_id').count()['Age'].std())
print("All Before 2 min", cond.groupby('person_id').count()['Age'].min())
cond1=cond[cond['person_id'].isin(train_ids)]
print("Train Before 2 max", cond1.groupby('person_id').count()['Age'].max())
print("Train Before 2 mean", cond1.groupby('person_id').count()['Age'].mean())
print("Train Before 2 std", cond1.groupby('person_id').count()['Age'].std())
print("Train Before 2 min", cond1.groupby('person_id').count()['Age'].min())
cond1=cond[cond['person_id'].isin(test_ids)]
print("Test Before 2 max", cond1.groupby('person_id').count()['Age'].max())
print("Test Before 2 mean", cond1.groupby('person_id').count()['Age'].mean())
print("Test Before 2 std", cond1.groupby('person_id').count()['Age'].std())
print("Test Before 2 min", cond1.groupby('person_id').count()['Age'].min())


cond=pd.read_csv('data/4/meas_after2.csv',header=0)
cond=cond[cond['person_id'].isin(all_ids)]
print(cond.head())
cols_to_std=cond.columns
print(cols_to_std)
# del cond[3013762]
# del cond[3023540]
# del cond[40762638]
# del cond[3001537]
# del cond[3038553]
cols_to_std=cond.columns
print(cols_to_std)
cols_to_std=cols_to_std[2:]
cond=pd.melt(cond, id_vars=['person_id','Age'], value_vars=cols_to_std)
print(cond.head())
cond['value']=cond['value'].astype(float)
cond['value']=cond['value'].astype(int)
cond=cond[cond['value']!=0]
cond=cond[cond['value']!='0']
print(cond.head())
del cond['value']
cond=cond.drop_duplicates()
print("All After 2 max", cond.groupby('person_id').count()['Age'].max())
print("All After 2 mean", cond.groupby('person_id').count()['Age'].mean())
print("All After 2 std", cond.groupby('person_id').count()['Age'].std())
print("All After 2 min", cond.groupby('person_id').count()['Age'].min())
cond1=cond[cond['person_id'].isin(train_ids)]
print("Train After 2 max", cond1.groupby('person_id').count()['Age'].max())
print("Train After 2 mean", cond1.groupby('person_id').count()['Age'].mean())
print("Train After 2 std", cond1.groupby('person_id').count()['Age'].std())
print("Train After 2 min", cond1.groupby('person_id').count()['Age'].min())
cond1=cond[cond['person_id'].isin(test_ids)]
print("Test After 2 max", cond1.groupby('person_id').count()['Age'].max())
print("Test After 2 mean", cond1.groupby('person_id').count()['Age'].mean())
print("Test After 2 std", cond1.groupby('person_id').count()['Age'].std())
print("Test After 2 min", cond1.groupby('person_id').count()['Age'].min())

In [ ]:

med=pd.read_csv('ATC10.csv',header=0)
med_ids=pd.read_csv('data/5/feat_meds.csv',header=0)
med=med[med['mapped_concept_name'].isin(med_ids['mapped_concept_name'].unique())]
med=med[med['Age']<=24]
del med['concept_id']
del med['mapped_concept_id']
med=med.drop_duplicates()
med=med.rename(columns={'mapped_concept_name':'value'})
med=med[['person_id','Age','value']]
print(med.head())
del med['Age']
cond=med

print(cond['value'].nunique())


cond=cond.drop_duplicates()
cond=cond[cond['person_id'].isin(all_ids)]
print("All Before 2 max", cond.groupby('person_id').count()['value'].max())
print("All Before 2 mean", cond.groupby('person_id').count()['value'].mean())
print("All Before 2 std", cond.groupby('person_id').count()['value'].std())
print("All Before 2 min", cond.groupby('person_id').count()['value'].min())
cond1=cond[cond['person_id'].isin(train_ids)]
print("Train Before 2 max", cond1.groupby('person_id').count()['value'].max())
print("Train Before 2 mean", cond1.groupby('person_id').count()['value'].mean())
print("Train Before 2 std", cond1.groupby('person_id').count()['value'].std())
print("Train Before 2 min", cond1.groupby('person_id').count()['value'].min())
cond1=cond[cond['person_id'].isin(test_ids)]
print("Test Before 2 max", cond1.groupby('person_id').count()['value'].max())
print("Test Before 2 mean", cond1.groupby('person_id').count()['value'].mean())
print("Test Before 2 std", cond1.groupby('person_id').count()['value'].std())
print("Test Before 2 min", cond1.groupby('person_id').count()['value'].min())


med=pd.read_csv('ATC10.csv',header=0)
med_ids=pd.read_csv('data/5/feat_meds.csv',header=0)
med=med[med['mapped_concept_name'].isin(med_ids['mapped_concept_name'].unique())]
med=med[med['Age']>24]
med=med[med['Age']<=120]
del med['concept_id']
del med['mapped_concept_id']
med=med.drop_duplicates()
med=med.rename(columns={'mapped_concept_name':'value'})
med=med[['person_id','Age','value']]
med.head()
del med['Age']
cond=med
cond=cond.drop_duplicates()
cond=cond[cond['person_id'].isin(all_ids)]
print("All After 2 max", cond.groupby('person_id').count()['value'].max())
print("All After 2 mean", cond.groupby('person_id').count()['value'].mean())
print("All After 2 min", cond.groupby('person_id').count()['value'].min())
cond1=cond[cond['person_id'].isin(train_ids)]
print("Train After 2 max", cond1.groupby('person_id').count()['value'].max())
print("Train After 2 mean", cond1.groupby('person_id').count()['value'].mean())
print("Train After 2 STd", cond1.groupby('person_id').count()['value'].std())
print("Train After 2 min", cond1.groupby('person_id').count()['value'].min())
cond1=cond[cond['person_id'].isin(test_ids)]
print("Test After 2 max", cond1.groupby('person_id').count()['value'].max())
print("Test After 2 mean", cond1.groupby('person_id').count()['value'].mean())
print("Test After 2 std", cond1.groupby('person_id').count()['value'].std())
print("Test After 2 min", cond1.groupby('person_id').count()['value'].min())

In [ ]:
cond=pd.read_csv('cond_cohort.csv',header=0)
cond=cond[cond['Age']<=24]
del cond['visit_occurrence_id']
del cond['fact_id']
del cond['Sex']
cond=cond.drop_duplicates()
cond=cond.rename(columns={'fact':'value'})
cond=cond[['person_id','Age','value']]
cond.head()

cond=cond.drop_duplicates()
cond=cond[cond['person_id'].isin(all_ids)]
print("All Before 2 max", cond.groupby('person_id').count()['Age'].max())
print("All Before 2 mean", cond.groupby('person_id').count()['Age'].mean())
print("All Before 2 min", cond.groupby('person_id').count()['Age'].min())
cond1=cond[cond['person_id'].isin(train_ids)]
print("Train Before 2 max", cond1.groupby('person_id').count()['Age'].max())
print("Train Before 2 mean", cond1.groupby('person_id').count()['Age'].mean())
print("Train Before 2 std", cond1.groupby('person_id').count()['Age'].std())
print("Train Before 2 min", cond1.groupby('person_id').count()['Age'].min())
cond1=cond[cond['person_id'].isin(test_ids)]
print("Test Before 2 max", cond1.groupby('person_id').count()['Age'].max())
print("Test Before 2 mean", cond1.groupby('person_id').count()['Age'].mean())
print("Test Before 2 std", cond1.groupby('person_id').count()['Age'].std())
print("Test Before 2 min", cond1.groupby('person_id').count()['Age'].min())


In [ ]:
cond=pd.read_csv('data/4/demo.csv',header=0)
cond=cond.drop_duplicates()
cond=cond[cond['person_id'].isin(all_ids)]
print("Race",cond.Race_label.value_counts())
print("Eth",cond.Eth_label.value_counts())
print("Payer",cond.payer_y.value_counts())
print("Gender",cond.Sex.value_counts())


cond1=cond[cond['person_id'].isin(train_ids)]
print("Race",cond1.Race_label.value_counts())
print("Eth",cond1.Eth_label.value_counts())
print("Payer",cond1.payer_y.value_counts())
print("Gender",cond1.Sex.value_counts())

cond1=cond[cond['person_id'].isin(test_ids)]
print("Race",cond1.Race_label.value_counts())
print("Eth",cond1.Eth_label.value_counts())
print("Payer",cond1.payer_y.value_counts())
print("Gender",cond1.Sex.value_counts())


In [ ]:
bmi=pd.read_csv('data/4/bmi_02.csv',header=0)
bmi=bmi[bmi['person_id'].isin(all_ids)]
bmi['label']=0
bmi.loc[(bmi['24']>0) & (bmi['24']<=5),'label']=1
bmi.loc[(bmi['24']>=5) & (bmi['24']<85),'label']=2
bmi.loc[(bmi['24']>=85) & (bmi['24']<95),'label']=3
bmi.loc[bmi['24']>=95,'label']=4
print(bmi.label.value_counts())

bmi1=bmi[bmi['person_id'].isin(train_ids)]
print(bmi1.label.value_counts())
bmi1=bmi[bmi['person_id'].isin(test_ids)]
print(bmi1.label.value_counts())

In [ ]:
bmi=pd.read_csv('data/geo/bmi_2_train.csv',header=0)
print(bmi.value.value_counts())

In [ ]:
bmi=pd.read_csv('data/geo/bmi_2_test.csv',header=0)
print(bmi.value.value_counts())

### Combining all info before 2

In [ ]:
final_ids=pd.read_csv('data/4/tt/all_id.csv',header=0)

In [ ]:
med=pd.read_csv('ATC10.csv',header=0)
med_ids=pd.read_csv('data/5/feat_meds.csv',header=0)
med=med[med['mapped_concept_name'].isin(med_ids['mapped_concept_name'].unique())]
med=med[med['Age']<=24]
del med['concept_id']
del med['mapped_concept_id']
med=med.drop_duplicates()
med=med.rename(columns={'mapped_concept_name':'value'})
med=med[['person_id','Age','value']]
med.head()

In [ ]:
med.value.nunique()

In [ ]:
cond=pd.read_csv('cond_cohort.csv',header=0)
cond=cond[cond['Age']<=24]
del cond['visit_occurrence_id']
del cond['fact_id']
del cond['Sex']
cond=cond.drop_duplicates()
cond=cond.rename(columns={'fact':'value'})
cond=cond[['person_id','Age','value']]
cond.head()

In [ ]:
bmi_pct=pd.read_csv('data/4/bmi_pct_enc_02.csv',header=0)
bmi_pct=bmi_pct.rename(columns={'variable':'Age'})
bmi_pct['value'] = 'BMIp_Change_' + bmi_pct['value'].astype(str)
bmi=pd.read_csv('data/4/bmi_02.csv',header=0)
bmi['Age']=24
bmi['value']=0
bmi.loc[(bmi['24']>0) & (bmi['24']<=5),'value']='Underweight'
bmi.loc[(bmi['24']>=5) & (bmi['24']<85),'value']='Normal'
bmi.loc[(bmi['24']>=85) & (bmi['24']<95),'value']='Overweight'
bmi.loc[bmi['24']>=95,'value']='Obesity'
bmi=bmi[['person_id','Age','value']]

In [ ]:
bmi_pct

In [ ]:
bmi

In [ ]:
bmi.to_csv('data/5/bmi_label_2.csv',index=False)

In [ ]:
meas=pd.read_csv('data/4/meas_02_5.csv',header=0)
del meas['variable']
meas.head()

In [ ]:
final_input=pd.concat([cond,meas],axis=0)
final_input=pd.concat([final_input,med],axis=0)
final_input=pd.concat([final_input,bmi_pct],axis=0)
final_input=pd.concat([final_input,bmi],axis=0)
final_input=final_input[final_input['person_id'].isin(final_ids['person_id'])]
final_input=final_input.sort_values(by=['person_id','Age'])
final_input.groupby('person_id').count()['Age'].max()

In [ ]:
lengths=final_input['person_id'].value_counts().reset_index()
lengths=lengths.sort_values(by=['index'])
lengths['index'].nunique()
lengths.to_csv('data/5/enc_lengths.csv',index=False)

In [ ]:
miss=final_input['person_id'].value_counts().reset_index()
miss['person_id']=660-miss['person_id']
dummy=pd.DataFrame({'person_id': miss['index'].repeat(miss['person_id'])})
final_input2=pd.concat([final_input,dummy],axis=0)
final_input2=final_input2.sort_values(by=['person_id','Age'])
final_input2=final_input2.fillna(0)

In [ ]:
final_input2.person_id.nunique()

In [ ]:
final_input2.to_csv('data/5/enc_data.csv',index=False)

In [ ]:
final_input2

### COMBINING DEC

In [ ]:
final_ids=pd.read_csv('data/4/tt/all_id.csv',header=0)
cond=pd.read_csv('data/4/cond_after2.csv',header=0)
meas=pd.read_csv('data/4/meas_after2.csv',header=0)
bmi=pd.read_csv('data/4/bmiPad_after2.csv',header=0)

In [ ]:
bmi.loc[(bmi['24']>0) & (bmi['24']<=5),'value']='Underweight'
bmi.loc[(bmi['24']>=5) & (bmi['24']<85),'value']='Normal'
bmi.loc[(bmi['24']>=85) & (bmi['24']<95),'value']='Overweight'
bmi.loc[bmi['24']>=95,'value']='Obesity'

In [ ]:
final_input=pd.merge(cond,meas,how='outer',on=['person_id','Age'])
final_input=final_input[final_input['person_id'].isin(final_ids['person_id'])]
final_input=final_input.sort_values(by=['person_id','Age'])
final_input=final_input.fillna(0)
final_input.person_id.nunique()

### CREATE DICTIONARY

In [ ]:
final_ids=pd.read_csv('data/4/tt/all_id.csv',header=0)
enc=pd.read_csv('data/5/enc_data.csv',header=0)
demo=pd.read_csv('data/5/tt/demo_all.csv',header=0)
cond=pd.read_csv('cond_cohort.csv',header=0)
cond=cond[cond['person_id'].isin(final_ids['person_id'])]
cond=cond[cond['Age']<=120]
del cond['visit_occurrence_id']
del cond['fact_id']
del cond['Sex']
cond=cond.drop_duplicates()

med=pd.read_csv('ATC10.csv',header=0)
med_ids=pd.read_csv('data/5/feat_meds.csv',header=0)
med=med[med['mapped_concept_name'].isin(med_ids['mapped_concept_name'].unique())]
med=med[med['Age']<=120]
del med['concept_id']
del med['mapped_concept_id']
med=med.drop_duplicates()
med=med.rename(columns={'mapped_concept_name':'value'})
med=med[['person_id','Age','value']]

In [ ]:
meas=pd.read_csv('data/4/meas_after2.csv',header=0)
cond_dec=pd.read_csv('data/4/cond_after2.csv',header=0)
med_dec=pd.read_csv('data/5/meds_after2_reduced.csv',header=0)
cond_dec=cond_dec[cond_dec['person_id'].isin(final_ids['person_id'])]
meas=meas[meas['person_id'].isin(final_ids['person_id'])]
med_dec=med_dec[med_dec['person_id'].isin(final_ids['person_id'])]

In [ ]:
cols_to_std=meas.columns
cols_to_std=cols_to_std[2:]
meas=pd.melt(meas, id_vars=['person_id','Age'], value_vars=cols_to_std)
meas=meas[meas['value']!='0']
meas=meas.drop_duplicates()


cols_to_std=cond_dec.columns
cols_to_std=cols_to_std[:-2]
cond_dec=pd.melt(cond_dec, id_vars=['person_id','Age'], value_vars=cols_to_std)
cond_dec=cond_dec[cond_dec['value']!=0]
#del cond['value']
cond_dec=cond_dec.drop_duplicates()

cols_to_std=med_dec.columns
cols_to_std=cols_to_std[:-2]
med_dec=pd.melt(med_dec, id_vars=['person_id','Age'], value_vars=cols_to_std)
med_dec=med_dec[med_dec['value']!=0]
#del cond['value']
med_dec=med_dec.drop_duplicates()

In [ ]:
med_dec.head()

In [ ]:
cond['fact'].unique()

In [ ]:
feat=list(enc['value'].unique())
feat.extend(list(cond['fact'].unique()))
feat.extend(list(cond_dec['variable'].unique()))
feat.extend(list(med_dec['variable'].unique()))
feat.extend(list(meas['value'].unique()))
# feat.extend(list(demo['Eth_label'].unique()))
# feat.extend(list(demo['Sex'].unique()))
# feat.extend(list(demo['Race_label'].unique()))
# feat.extend(list(demo['payer_y'].unique()))
feat.extend([0])
feat=list(set(feat))

In [ ]:
len(feat)

In [ ]:
l_dict = dict(zip(set(feat), range(len(feat))))
l_dict['NI']=0
l_dict['0']=0
l_dict

In [ ]:
l_dict

In [ ]:
l_dict['BMIp_Change_4']

In [ ]:
import pickle
with open("data/5/featVocab", 'wb') as fp:
        pickle.dump(l_dict, fp)

In [ ]:
feat=list(demo['Eth_label'].unique())
feat.extend(list(demo['Sex'].unique()))
feat.extend(list(demo['Race_label'].unique()))
feat.extend(list(demo['payer_y'].unique()))
feat.extend(list(demo['coi'].unique()))
feat.extend([0])
feat=list(set(feat))
feat

In [ ]:
d_dict = dict(zip(set(feat), range(len(feat))))
d_dict['NI']=0
d_dict['Asian']=4
d_dict['0']=0
d_dict

In [ ]:
len(d_dict)

In [ ]:
import pickle
with open("data/5/demoVocab", 'wb') as fp:
        pickle.dump(d_dict, fp)

In [ ]:
486+20

### Train and test

In [ ]:
train_id=pd.read_csv('data/4/train_ids.csv',header=0)
test_id=pd.read_csv('data/4/test_ids.csv',header=0)
mask=pd.read_csv('data/4/bmiMask_after2.csv',header=0)
bmi=pd.read_csv('data/4/bmi_02.csv',header=0)
all_ids=pd.concat([train_id,test_id],axis=0)
all_ids.shape

In [ ]:
print(bmi.person_id.nunique())
print(mask.person_id.nunique())

In [ ]:
final_ids=pd.merge(bmi['person_id'],mask['person_id'],how='inner',on='person_id')
final_ids.person_id.nunique()

In [ ]:
final_ids=pd.merge(all_ids['person_id'],mask['person_id'],how='inner',on='person_id')
final_ids=pd.merge(final_ids['person_id'],bmi['person_id'],how='inner',on='person_id')
final_ids.person_id.nunique()

In [ ]:
train_id=train_id[train_id['person_id'].isin(final_ids['person_id'])]
train_id.shape

In [ ]:
test_id=test_id[test_id['person_id'].isin(final_ids['person_id'])]
test_id.shape

In [ ]:
train_id.to_csv('data/4/tt/train_id.csv',index=False)
test_id.to_csv('data/4/tt/test_id.csv',index=False)
final_ids.to_csv('data/4/tt/all_id.csv',index=False)

### GEO SPLIT

In [ ]:
train_id=pd.read_csv('data/4/tt/all_id.csv',header=0)
test_id=pd.read_csv('fl.csv',header=0)
test_id=train_id[train_id['person_id'].isin(list(test_id['person_id'].unique()))]
train_id=train_id[~train_id['person_id'].isin(list(test_id['person_id'].unique()))]

In [ ]:
train_id.shape

In [ ]:
test_id.shape

In [ ]:
demo=pd.read_csv('data/5/tt/demo_all.csv',header=0)
demo1=demo[demo['person_id'].isin(train_id['person_id'])]
demo1=demo1.sort_values(by='person_id')
demo1.to_csv('data/geo/demo_train.csv',index=False)
demo1=demo[demo['person_id'].isin(test_id['person_id'])]
demo1=demo1.sort_values(by='person_id')
demo1.to_csv('data/geo/demo_test.csv',index=False)
demo.head()

In [ ]:
demo=pd.read_csv('data/geo/demo_test.csv',header=0)
demo=demo.sort_values(by='person_id')
demo['Eth_dict']=[d_dict[x] for x in demo['Eth_label']]
demo['Race_dict']=[d_dict[x] for x in demo['Race_label']]
demo['Sex_dict']=[d_dict[x] for x in demo['Sex']]
demo['Payer_dict']=[d_dict[x] for x in demo['payer_y']]
demo['COI_dict']=[d_dict[x] for x in demo['coi']]
demo.to_csv('data/geo/demo_test.csv',index=False)

demo=pd.read_csv('data/geo/demo_train.csv',header=0)
demo=demo.sort_values(by='person_id')
demo['Eth_dict']=[d_dict[x] for x in demo['Eth_label']]
demo['Race_dict']=[d_dict[x] for x in demo['Race_label']]
demo['Sex_dict']=[d_dict[x] for x in demo['Sex']]
demo['Payer_dict']=[d_dict[x] for x in demo['payer_y']]
demo['COI_dict']=[d_dict[x] for x in demo['coi']]
demo.to_csv('data/geo/demo_train.csv',index=False)

In [ ]:
enc=pd.read_csv('data/5/enc_data.csv',header=0)
enc_len=pd.read_csv('data/5/enc_lengths.csv',header=0)

In [ ]:
enc_len1=enc_len[enc_len['index'].isin(train_id['person_id'])]
enc_len1=enc_len1.sort_values(by=['index'])
enc_len1.to_csv('data/geo/enc_len_train.csv',index=False)
enc_len1=enc_len[enc_len['index'].isin(test_id['person_id'])]
enc_len1=enc_len1.sort_values(by=['index'])
enc_len1.to_csv('data/geo/enc_len_test.csv',index=False)

In [ ]:
enc['feat_dict']=[l_dict[x] for x in enc['value']]
enc['age_dict']=[a_dict[x] for x in enc['Age']]

enc1=enc[enc['person_id'].isin(train_id['person_id'])]
enc1.to_csv('data/geo/enc_train.csv',index=False)
enc1=enc[enc['person_id'].isin(test_id['person_id'])]
enc1.to_csv('data/geo/enc_test.csv',index=False)

In [ ]:
bmi=pd.read_csv('data/4/bmi_label_2.csv',header=0)
del bmi['Age']
bmi=bmi.sort_values(by='person_id')

bmi1=bmi[bmi['person_id'].isin(train_id['person_id'])]
print(bmi1.person_id.nunique())
bmi1.to_csv('data/geo/bmi_2_train.csv',index=False)
bmi1=bmi[bmi['person_id'].isin(test_id['person_id'])]
print(bmi1.person_id.nunique())
bmi1.to_csv('data/geo/bmi_2_test.csv',index=False)

In [ ]:
labels=pd.read_csv('data/4/bmiOb_after2.csv',header=0)
labels=labels.sort_values(by='person_id')
cols_to_std=labels.columns
cols_to_std=cols_to_std[1:]
labels=pd.melt(labels, id_vars=['person_id'], value_vars=cols_to_std)
labels['variable']=labels['variable'].astype(int)
labels=labels.sort_values(by=['person_id','variable'])
labels=labels.rename(columns={'variable':'Age'})
labels1=labels[labels['person_id'].isin(train_id['person_id'])]
labels1=labels1.sort_values(by=['person_id','Age'])
print(labels1.person_id.nunique())
labels1.to_csv('data/geo/labels_train.csv',index=False)
labels1=labels[labels['person_id'].isin(test_id['person_id'])]
labels1=labels1.sort_values(by=['person_id','Age'])
print(labels1.person_id.nunique())
labels1.to_csv('data/geo/labels_test.csv',index=False)

In [ ]:
mask=pd.read_csv('data/4/bmiMask_after2.csv',header=0)
mask=mask.sort_values(by='person_id')
cols_to_std=mask.columns
cols_to_std=cols_to_std[1:]
mask=pd.melt(mask, id_vars=['person_id'], value_vars=cols_to_std)
mask['variable']=mask['variable'].astype(int)
mask=mask.sort_values(by=['person_id','variable'])
mask=mask.rename(columns={'variable':'Age'})

mask1=mask[mask['person_id'].isin(train_id['person_id'])]
mask1=mask1.sort_values(by=['person_id','Age'])
print(mask1.person_id.nunique())
mask1.to_csv('data/geo/mask_train.csv',index=False)
mask1=mask[mask['person_id'].isin(test_id['person_id'])]
mask1=mask1.sort_values(by=['person_id','Age'])
print(mask1.person_id.nunique())
mask1.to_csv('data/geo/mask_test.csv',index=False)


In [ ]:
final=pd.read_csv('data/5/tt/dec_all.csv',header=0)
final1=final[final['person_id'].isin(train_id['person_id'])]
final1=final1.sort_values(by=['person_id','age_dict'])
print(final1.person_id.nunique())
final1.to_csv('data/geo/dec_train.csv',index=False)
final1=final[final['person_id'].isin(test_id['person_id'])]
final1=final1.sort_values(by=['person_id','age_dict'])
print(final1.person_id.nunique())
final1.to_csv('data/geo/dec_test.csv',index=False)

In [ ]:
test_id.to_csv('./data/geo/test_id.csv',index=False)
train_id.to_csv('./data/geo/train_id.csv',index=False)

### Demo split

In [ ]:
train_id=pd.read_csv('data/4/tt/train_id.csv',header=0)
test_id=pd.read_csv('data/4/tt/test_id.csv',header=0)
final_ids=pd.read_csv('data/4/tt/all_id.csv',header=0)

In [ ]:
demo=pd.read_csv('data/4/demo.csv',header=0)
demo=demo.fillna('NI')
demo=demo[demo['person_id'].isin(final_ids['person_id'])]
demo=demo[['person_id','Eth_label','Race_label','payer_y','Sex']]
demo.to_csv('data/4/tt/demo_all.csv',index=False)

In [ ]:
demo=pd.read_csv('data/5/tt/demo_all.csv',header=0)
demo1=demo[demo['person_id'].isin(train_id['person_id'])]
demo1=demo1.sort_values(by='person_id')
demo1.to_csv('data/5/tt/demo_train.csv',index=False)
demo1=demo[demo['person_id'].isin(test_id['person_id'])]
demo1=demo1.sort_values(by='person_id')
demo1.to_csv('data/5/tt/demo_test.csv',index=False)
demo.head()

### FINAL PREP ENC

In [ ]:
enc=pd.read_csv('data/5/enc_data.csv',header=0)
enc_len=pd.read_csv('data/5/enc_lengths.csv',header=0)
train_id=pd.read_csv('data/4/tt/train_id.csv',header=0)
test_id=pd.read_csv('data/4/tt/test_id.csv',header=0)
final_ids=pd.read_csv('data/4/tt/all_id.csv',header=0)

In [ ]:
enc_len1=enc_len[enc_len['index'].isin(train_id['person_id'])]
enc_len1=enc_len1.sort_values(by=['index'])
enc_len1.to_csv('data/5/tt/enc_len_train.csv',index=False)
enc_len1=enc_len[enc_len['index'].isin(test_id['person_id'])]
enc_len1=enc_len1.sort_values(by=['index'])
enc_len1.to_csv('data/5/tt/enc_len_test.csv',index=False)

In [ ]:
import pickle
with open("ageVocab", 'rb') as fp:
                a_dict=pickle.load(fp)
        
with open("data/5/featVocab", 'rb') as fp:
                l_dict=pickle.load(fp)
        
with open("data/5/demoVocab", 'rb') as fp:
                d_dict=pickle.load(fp)

In [ ]:
enc['feat_dict']=[l_dict[x] for x in enc['value']]
enc['age_dict']=[a_dict[x] for x in enc['Age']]

enc1=enc[enc['person_id'].isin(train_id['person_id'])]
enc1.to_csv('data/5/tt/enc_train.csv',index=False)
enc1=enc[enc['person_id'].isin(test_id['person_id'])]
enc1.to_csv('data/5/tt/enc_test.csv',index=False)

In [ ]:
demo=pd.read_csv('data/5/tt/demo_test.csv',header=0)
demo=demo.sort_values(by='person_id')
demo['Eth_dict']=[d_dict[x] for x in demo['Eth_label']]
demo['Race_dict']=[d_dict[x] for x in demo['Race_label']]
demo['Sex_dict']=[d_dict[x] for x in demo['Sex']]
demo['Payer_dict']=[d_dict[x] for x in demo['payer_y']]
demo['COI_dict']=[d_dict[x] for x in demo['coi']]
demo.to_csv('data/5/tt/demo_test.csv',index=False)

demo=pd.read_csv('data/5/tt/demo_train.csv',header=0)
demo=demo.sort_values(by='person_id')
demo['Eth_dict']=[d_dict[x] for x in demo['Eth_label']]
demo['Race_dict']=[d_dict[x] for x in demo['Race_label']]
demo['Sex_dict']=[d_dict[x] for x in demo['Sex']]
demo['Payer_dict']=[d_dict[x] for x in demo['payer_y']]
demo['COI_dict']=[d_dict[x] for x in demo['coi']]
demo.to_csv('data/5/tt/demo_train.csv',index=False)

In [ ]:
bmi=pd.read_csv('data/4/bmi_label_2.csv',header=0)
del bmi['Age']
bmi=bmi.sort_values(by='person_id')

bmi1=bmi[bmi['person_id'].isin(train_id['person_id'])]
print(bmi1.person_id.nunique())
bmi1.to_csv('data/5/tt/bmi_2_train.csv',index=False)
bmi1=bmi[bmi['person_id'].isin(test_id['person_id'])]
print(bmi1.person_id.nunique())
bmi1.to_csv('data/5/tt/bmi_2_test.csv',index=False)


### FINAL PREP DEC

In [ ]:
train_id=pd.read_csv('data/4/tt/train_id.csv',header=0)
test_id=pd.read_csv('data/4/tt/test_id.csv',header=0)
final_ids=pd.read_csv('data/4/tt/all_id.csv',header=0)

In [ ]:
import pickle
with open("ageVocab", 'rb') as fp:
                a_dict=pickle.load(fp)
        
with open("data/5/featVocab", 'rb') as fp:
                l_dict=pickle.load(fp)

In [ ]:
labels=pd.read_csv('data/4/bmiOb_after2.csv',header=0)
labels=labels.sort_values(by='person_id')
cols_to_std=labels.columns
cols_to_std=cols_to_std[1:]
labels=pd.melt(labels, id_vars=['person_id'], value_vars=cols_to_std)
labels['variable']=labels['variable'].astype(int)
labels=labels.sort_values(by=['person_id','variable'])
labels=labels.rename(columns={'variable':'Age'})
labels1=labels[labels['person_id'].isin(train_id['person_id'])]
labels1=labels1.sort_values(by=['person_id','Age'])
print(labels1.person_id.nunique())
labels1.to_csv('data/4/tt/labels_train.csv',index=False)
labels1=labels[labels['person_id'].isin(test_id['person_id'])]
labels1=labels1.sort_values(by=['person_id','Age'])
print(labels1.person_id.nunique())
labels1.to_csv('data/4/tt/labels_test.csv',index=False)

In [ ]:
mask=pd.read_csv('data/4/bmiMask_after2.csv',header=0)
mask=mask.sort_values(by='person_id')
cols_to_std=mask.columns
cols_to_std=cols_to_std[1:]
mask=pd.melt(mask, id_vars=['person_id'], value_vars=cols_to_std)
mask['variable']=mask['variable'].astype(int)
mask=mask.sort_values(by=['person_id','variable'])
mask=mask.rename(columns={'variable':'Age'})

mask1=mask[mask['person_id'].isin(train_id['person_id'])]
mask1=mask1.sort_values(by=['person_id','Age'])
print(mask1.person_id.nunique())
mask1.to_csv('data/4/tt/mask_train.csv',index=False)
mask1=mask[mask['person_id'].isin(test_id['person_id'])]
mask1=mask1.sort_values(by=['person_id','Age'])
print(mask1.person_id.nunique())
mask1.to_csv('data/4/tt/mask_test.csv',index=False)


In [ ]:
mask1.head()

In [ ]:
meas=pd.read_csv('data/4/meas_after2.csv',header=0)

meas=meas[meas['person_id'].isin(final_ids['person_id'])]
cols_to_std=meas.columns
cols_to_std=cols_to_std[2:]
meas=pd.melt(meas, id_vars=['person_id','Age'], value_vars=cols_to_std)
meas['feat_dict']=[l_dict[x] for x in meas['value']]
meas['age_dict']=[a_dict[x] for x in meas['Age']]


meas=meas.pivot_table(index=['person_id','age_dict'],columns='variable',values='feat_dict', aggfunc='first')
meas=meas.reset_index()
meas.head()

In [ ]:
meas.columns

In [ ]:
cond_dec=pd.read_csv('data/4/cond_after2.csv',header=0)
cond_dec=cond_dec[cond_dec['person_id'].isin(final_ids['person_id'])]
cond_dec=cond_dec.loc[:, (cond_dec != 0).any(axis=0)]
cols_to_std=cond_dec.columns
cols_to_std=cols_to_std[:-2]

cond_dec=pd.melt(cond_dec, id_vars=['person_id','Age'], value_vars=cols_to_std)

cond_dec=cond_dec.drop_duplicates()
cond_dec['feat_dict']=[l_dict[x] for x in cond_dec['variable']]
cond_dec['age_dict']=[a_dict[x] for x in cond_dec['Age']]

cond_dec=cond_dec.pivot_table(index=['person_id','age_dict'],columns='variable',values='feat_dict', aggfunc='first')
cond_dec=cond_dec.reset_index()
cond_dec.head()

In [ ]:
cond_dec.shape

In [ ]:
med_dec=pd.read_csv('data/5/meds_after2_reduced.csv',header=0)
med_dec=med_dec[med_dec['person_id'].isin(final_ids['person_id'])]
med_dec=med_dec.loc[:, (med_dec != 0).any(axis=0)]
cols_to_std=med_dec.columns
cols_to_std=cols_to_std[:-2]

med_dec=pd.melt(med_dec, id_vars=['person_id','Age'], value_vars=cols_to_std)

med_dec=med_dec.drop_duplicates()
med_dec['feat_dict']=[l_dict[x] for x in med_dec['variable']]
med_dec['age_dict']=[a_dict[x] for x in med_dec['Age']]

med_dec=med_dec.pivot_table(index=['person_id','age_dict'],columns='variable',values='feat_dict', aggfunc='first')
med_dec=med_dec.reset_index()
med_dec.head()

In [ ]:
cond_dec.person_id.nunique()

In [ ]:
meas.person_id.nunique()

In [ ]:
med_dec.person_id.nunique()

In [ ]:
meas.shape

In [ ]:
med_dec.shape

In [ ]:
final=pd.merge(cond_dec,meas,on=['person_id','age_dict'],how='outer')
final=pd.merge(final,med_dec,on=['person_id','age_dict'],how='outer')

In [ ]:
final.shape

In [ ]:
final.person_id.nunique()

In [ ]:
final.head()

In [ ]:
final=pd.read_csv('data/5/tt/dec_all.csv',header=0)

In [ ]:
final.to_csv('data/5/tt/dec_all.csv',index=False)
final1=final[final['person_id'].isin(train_id['person_id'])]
final1=final1.sort_values(by=['person_id','age_dict'])
print(final1.person_id.nunique())
final1.to_csv('data/5/tt/dec_train.csv',index=False)
final1=final[final['person_id'].isin(test_id['person_id'])]
final1=final1.sort_values(by=['person_id','age_dict'])
print(final1.person_id.nunique())
final1.to_csv('data/5/tt/dec_test.csv',index=False)

### FINAl VERIFICATION oF ID SORT

In [ ]:
enc_test=pd.read_csv('data/5/tt/enc_test.csv',header=0)
len_test=pd.read_csv('data/5/tt/enc_len_test.csv',header=0)
demo_test=pd.read_csv('data/5/tt/demo_test.csv',header=0)
bmi=pd.read_csv('data/5/tt/bmi_2_test.csv',header=0)

dec_test=pd.read_csv('data/5/tt/dec_test.csv',header=0)
labels_test=pd.read_csv('data/4/tt/labels_test.csv',header=0)
mask_test=pd.read_csv('data/4/tt/mask_test.csv',header=0)

In [ ]:
print(enc_test.person_id.nunique())
print(len_test['index'].nunique())
print(demo_test.person_id.nunique())
print(bmi.person_id.nunique())
print(dec_test.person_id.nunique())
print(labels_test.person_id.nunique())
print(mask_test.person_id.nunique())

In [ ]:
enc_test

In [ ]:
demo_test

In [ ]:
len_test.head()

In [ ]:
bmi.head()

In [ ]:
list(dec_test.person_id)

In [ ]:
dec_test.head()

In [ ]:
labels_test.head()

In [ ]:
list(labels_test.person_id)

In [ ]:
mask_test.head()

In [ ]:
list(mask_test.person_id)

In [ ]:
len(l_dict)

In [ ]:
len(d_dict)

### BMI Mapping

In [ ]:
fbmi=pd.read_csv('Female_BMI2.csv',header=0)
mbmi=pd.read_csv('Male_BMI2.csv',header=0)

In [ ]:
for row in range(fbmi.shape[0]):
    labs.loc[(labs.Sex=='Female') & (labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI<=fbmi.iloc[row,1]),'label']=3
    labs.loc[(labs.Sex=='Female') &(labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI>fbmi.iloc[row,1]) & (labs.BMI<=fbmi.iloc[row,2]),'label']=5
    labs.loc[(labs.Sex=='Female') & (labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI>fbmi.iloc[row,2]) & (labs.BMI<=fbmi.iloc[row,3]),'label']=10
    labs.loc[(labs.Sex=='Female') &(labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI>fbmi.iloc[row,3]) & (labs.BMI<=fbmi.iloc[row,4]),'label']=25
    labs.loc[(labs.Sex=='Female') & (labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI>fbmi.iloc[row,4]) & (labs.BMI<=fbmi.iloc[row,5]),'label']=50
    labs.loc[(labs.Sex=='Female') &(labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI>fbmi.iloc[row,5]) & (labs.BMI<=fbmi.iloc[row,6]),'label']=75
    labs.loc[(labs.Sex=='Female') & (labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI>fbmi.iloc[row,6]) & (labs.BMI<=fbmi.iloc[row,7]),'label']=85
    labs.loc[(labs.Sex=='Female') &(labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI>fbmi.iloc[row,7]) & (labs.BMI<=fbmi.iloc[row,8]),'label']=90
    labs.loc[(labs.Sex=='Female') & (labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI>fbmi.iloc[row,8]) & (labs.BMI<=fbmi.iloc[row,9]),'label']=95
    labs.loc[(labs.Sex=='Female') & (labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI>fbmi.iloc[row,9]) & (labs.BMI<=fbmi.iloc[row,10]),'label']=97
    labs.loc[(labs.Sex=='Female') & (labs.Age>=fbmi.iloc[row,0]) & (labs.Age<fbmi.iloc[row,0] + 1) & (labs.BMI>fbmi.iloc[row,10]),'label']=100
    

In [ ]:
for row in range(mbmi.shape[0]):
    labs.loc[(labs.Sex=='Male') & (labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI<=mbmi.iloc[row,1]),'label']=3
    labs.loc[(labs.Sex=='Male') &(labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI>mbmi.iloc[row,1]) & (labs.BMI<=mbmi.iloc[row,2]),'label']=5
    labs.loc[(labs.Sex=='Male') & (labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI>mbmi.iloc[row,2]) & (labs.BMI<=mbmi.iloc[row,3]),'label']=10
    labs.loc[(labs.Sex=='Male') &(labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI>mbmi.iloc[row,3]) & (labs.BMI<=mbmi.iloc[row,4]),'label']=25
    labs.loc[(labs.Sex=='Male') & (labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI>mbmi.iloc[row,4]) & (labs.BMI<=mbmi.iloc[row,5]),'label']=50
    labs.loc[(labs.Sex=='Male') &(labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI>mbmi.iloc[row,5]) & (labs.BMI<=mbmi.iloc[row,6]),'label']=75
    labs.loc[(labs.Sex=='Male') & (labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI>mbmi.iloc[row,6]) & (labs.BMI<=mbmi.iloc[row,7]),'label']=85
    labs.loc[(labs.Sex=='Male') &(labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI>mbmi.iloc[row,7]) & (labs.BMI<=mbmi.iloc[row,8]),'label']=90
    labs.loc[(labs.Sex=='Male') & (labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI>mbmi.iloc[row,8]) & (labs.BMI<=mbmi.iloc[row,9]),'label']=95
    labs.loc[(labs.Sex=='Male') & (labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI>mbmi.iloc[row,9]) & (labs.BMI<=mbmi.iloc[row,10]),'label']=97
    labs.loc[(labs.Sex=='Male') & (labs.Age>=mbmi.iloc[row,0]) & (labs.Age<mbmi.iloc[row,0] + 1) & (labs.BMI>mbmi.iloc[row,10]) ,'label']=100
    

In [ ]:
labs=bmi

In [ ]:
demo=pd.read_csv('demo2.csv',header=0)
labs=pd.merge(labs,demo[['person_id','Sex']],on='person_id',how='left')
labs['label']=0

In [ ]:
bmi=labs

In [ ]:
fbmi=pd.read_csv('femaleinfant2.csv',header=0)
mbmi=pd.read_csv('maleinfant2.csv',header=0)

In [ ]:
for row in range(fbmi.shape[0]):
    labs.loc[(labs.Sex=='Female') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=fbmi.iloc[row,0]) & (labs[3023540]<fbmi.iloc[row,0] + 1) & (labs[3013762]<=fbmi.iloc[row,1]),'label']=3
    labs.loc[(labs.Sex=='Female') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=fbmi.iloc[row,0]) & (labs[3023540]<fbmi.iloc[row,0] + 1) & (labs[3013762]>fbmi.iloc[row,1]) & (labs[3013762]<=fbmi.iloc[row,2]),'label']=5
    labs.loc[(labs.Sex=='Female') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=fbmi.iloc[row,0]) & (labs[3023540]<fbmi.iloc[row,0] + 1) & (labs[3013762]>fbmi.iloc[row,2]) & (labs[3013762]<=fbmi.iloc[row,3]),'label']=10
    labs.loc[(labs.Sex=='Female') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=fbmi.iloc[row,0]) & (labs[3023540]<fbmi.iloc[row,0] + 1) & (labs[3013762]>fbmi.iloc[row,3]) & (labs[3013762]<=fbmi.iloc[row,4]),'label']=25
    labs.loc[(labs.Sex=='Female') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=fbmi.iloc[row,0]) & (labs[3023540]<fbmi.iloc[row,0] + 1) & (labs[3013762]>fbmi.iloc[row,4]) & (labs[3013762]<=fbmi.iloc[row,5]),'label']=50
    labs.loc[(labs.Sex=='Female') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=fbmi.iloc[row,0]) & (labs[3023540]<fbmi.iloc[row,0] + 1) & (labs[3013762]>fbmi.iloc[row,5]) & (labs[3013762]<=fbmi.iloc[row,6]),'label']=75
    labs.loc[(labs.Sex=='Female') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=fbmi.iloc[row,0]) & (labs[3023540]<fbmi.iloc[row,0] + 1) & (labs[3013762]>fbmi.iloc[row,6]) & (labs[3013762]<=fbmi.iloc[row,7]),'label']=90
    labs.loc[(labs.Sex=='Female') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=fbmi.iloc[row,0]) & (labs[3023540]<fbmi.iloc[row,0] + 1) & (labs[3013762]>fbmi.iloc[row,7]) & (labs[3013762]<=fbmi.iloc[row,8]),'label']=95
    labs.loc[(labs.Sex=='Female') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=fbmi.iloc[row,0]) & (labs[3023540]<fbmi.iloc[row,0] + 1) & (labs[3013762]>fbmi.iloc[row,8]) & (labs[3013762]<=fbmi.iloc[row,9]),'label']=97
    labs.loc[(labs.Sex=='Female') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=fbmi.iloc[row,0]) & (labs[3023540]<fbmi.iloc[row,0] + 1) & (labs[3013762]>fbmi.iloc[row,9]),'label']=100

In [ ]:
for row in range(mbmi.shape[0]):
    labs.loc[(labs.Sex=='Male') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=mbmi.iloc[row,0]) & (labs[3023540]<mbmi.iloc[row,0] + 1) & (labs[3013762]<=mbmi.iloc[row,1]),'label']=3
    labs.loc[(labs.Sex=='Male') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=mbmi.iloc[row,0]) & (labs[3023540]<mbmi.iloc[row,0] + 1) & (labs[3013762]>mbmi.iloc[row,1]) & (labs[3013762]<=mbmi.iloc[row,2]),'label']=5
    labs.loc[(labs.Sex=='Male') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=mbmi.iloc[row,0]) & (labs[3023540]<mbmi.iloc[row,0] + 1) & (labs[3013762]>mbmi.iloc[row,2]) & (labs[3013762]<=mbmi.iloc[row,3]),'label']=10
    labs.loc[(labs.Sex=='Male') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=mbmi.iloc[row,0]) & (labs[3023540]<mbmi.iloc[row,0] + 1) & (labs[3013762]>mbmi.iloc[row,3]) & (labs[3013762]<=mbmi.iloc[row,4]),'label']=25
    labs.loc[(labs.Sex=='Male') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=mbmi.iloc[row,0]) & (labs[3023540]<mbmi.iloc[row,0] + 1) & (labs[3013762]>mbmi.iloc[row,4]) & (labs[3013762]<=mbmi.iloc[row,5]),'label']=50
    labs.loc[(labs.Sex=='Male') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=mbmi.iloc[row,0]) & (labs[3023540]<mbmi.iloc[row,0] + 1) & (labs[3013762]>mbmi.iloc[row,5]) & (labs[3013762]<=mbmi.iloc[row,6]),'label']=75
    labs.loc[(labs.Sex=='Male') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=mbmi.iloc[row,0]) & (labs[3023540]<mbmi.iloc[row,0] + 1) & (labs[3013762]>mbmi.iloc[row,6]) & (labs[3013762]<=mbmi.iloc[row,7]),'label']=90
    labs.loc[(labs.Sex=='Male') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=mbmi.iloc[row,0]) & (labs[3023540]<mbmi.iloc[row,0] + 1) & (labs[3013762]>mbmi.iloc[row,7]) & (labs[3013762]<=mbmi.iloc[row,8]),'label']=95
    labs.loc[(labs.Sex=='Male') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=mbmi.iloc[row,0]) & (labs[3023540]<mbmi.iloc[row,0] + 1) & (labs[3013762]>mbmi.iloc[row,8]) & (labs[3013762]<=mbmi.iloc[row,9]),'label']=97
    labs.loc[(labs.Sex=='Male') & (labs.Age>=0) & (labs.Age<24) & (labs[3023540]>=mbmi.iloc[row,0]) & (labs[3023540]<mbmi.iloc[row,0] + 1) & (labs[3013762]>mbmi.iloc[row,9]),'label']=100

In [ ]:
labs.head()

In [ ]:
labels=pd.read_csv('labels.csv',header=0)

In [ ]:
labels.head()

In [ ]:
labels[labels['obese']==0].count()

In [ ]:
labels[labels['obese']==1].count()

In [ ]:
pids=list((set(labels['person_id'].unique())).intersection(set(cond['person_id'].unique())))

In [ ]:
labels=labels[labels['person_id'].isin(pids)]
cond=cond[cond['person_id'].isin(pids)]

In [ ]:
labels[labels['obese']==0].count()

In [ ]:
labels[labels['obese']==1].count()

In [ ]:
cond.shape

In [ ]:
labels.shape

In [ ]:
print(cond['person_id'].nunique())
print(labels['person_id'].nunique())

In [ ]:
labels_ob=labels[labels['obese']==1]
labels_n=labels[labels['obese']==0]
cond_ob=cond[cond['person_id'].isin(labels_ob['person_id'].unique())]
cond_n=cond[cond['person_id'].isin(labels_n['person_id'].unique())]

In [ ]:
X_ob,xtest_ob,y_ob,ytest_ob=train_test_split(cond_ob, labels_ob, test_size=0.2, random_state=42)
X_n,xtest_n,y_n,ytest_n=train_test_split(cond_n, labels_n, test_size=0.2, random_state=42)

In [ ]:
X_ob2,_,y_ob2,_=train_test_split(X_ob, y_ob, test_size=0.5, random_state=42)

In [ ]:
X_ob=pd.concat([X_ob,X_ob2],axis=0)
y_ob=pd.concat([y_ob,y_ob2],axis=0)

In [ ]:
X_ob.shape

In [ ]:
X_n.shape

In [ ]:
X=pd.concat([X_ob,X_n],axis=0)
y=pd.concat([y_ob,y_n],axis=0)

In [ ]:
X.shape

In [ ]:
y.head()

In [ ]:
X_test=pd.concat([xtest_ob,xtest_n],axis=0)
y_test=pd.concat([ytest_ob,ytest_n],axis=0)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.DataFrame(load_breast_cancer()['data'],
columns=load_breast_cancer()['feature_names'])
df['y'] = load_breast_cancer()['target']
df.head(5)

In [ ]:
X,y = df.drop('y',axis=1),df.y
kf = KFold(n_splits=5,random_state=42,shuffle=True)
for train_index,val_index in kf.split(X):
    X_train,X_val = X.iloc[train_index],X.iloc[val_index],
    y_train,y_val = y.iloc[train_index],y.iloc[val_index],

In [ ]:
X_train

In [ ]:
del X['person_id']
del y['person_id']
del X_test['person_id']
del y_test['person_id']

In [ ]:
gradient_booster = GradientBoostingClassifier(learning_rate=0.1,max_depth=10,max_features="auto")
gradient_booster.get_params()

In [ ]:
gradient_booster.fit(X,y)
print(classification_report(y_test,gradient_booster.predict(X_test)))